In [1]:
import json
import pandas as pd
import re
from collections import defaultdict, Counter
from bs4 import BeautifulSoup

# 读取数据集
LABEL_PATH = './data_2/train/'
label_trainset = json.load(open(LABEL_PATH + 'label_trainset.json',encoding='utf8'))
instruction_trainset = json.load(open(LABEL_PATH + 'instruction_trainset.json',encoding='utf8'))
instruction_testA = json.load(open(LABEL_PATH + 'instruction_testA.json', encoding='utf8')) # A榜
instruction_testB = json.load(open(LABEL_PATH + 'instruction_testB_with_node.json', encoding='utf8')) # B榜

## 数据补充

- 根据官方给的站点数据补充训练数据

- 此处由于时间问题 补充了9个比较容易出现幻觉并且站点数据量比较多的站点


In [2]:
page_source = "中国裁判文书网"
import random
from random import choice
from datetime import datetime

def test_1():
    advs = ["", "请"]
    verbs = ["查找","查询","搜索","检索","查一下","检索一下","查询一下","查一查","检索一下","查找出"]
    data = {}
    data["审判程序s"] = ["民事一审"]
    data["案由s"] = "民事案由 刑事案由 劳动争议 物权纠纷 行政处罚 证券纠纷 财产保全 污染环境罪 侵权责任纠纷 海事海商纠纷 证券投资咨询纠纷 违法司法罚款赔偿".split()
    data["文书类型s"] = "判决书 裁定书 决定书 文书 调解书 通知书".split()
    data["法院名称s"] = "广东省深圳市中级人民法院 上海市金融法院 北京市第三中级人民法院 湖南省高级人民法院 江苏省高级人民法院 北京市高级人民法院 北京市第三中级人民法院 北京市第一中级人民法院 浙江省杭州市中级人民法院 北京市第三中级人民法院".split()
    data["案件类型s"] = "司法制裁案件 管辖案件 行政案件 国家赔偿与司法救助案件 民事案件 非诉保全审查案件 刑事案件 强制清算与破产案件 国际司法协助案件 非诉保全审查案件".split()
    data["当事人s"] = "凌孟夏 牛晨轩 马宏茂 陈义 卢嘉云 韦希 郎辉 洪嘉歆".split()
    data["律师s"] = "郑奇伟 贺涵易 邱斐 甄俊誉 孔博涛 宗彦 郑琛瑞 喻嘉悦".split()
    data["审判人员s"] = "郑念雁 卢明志 焦青易 陈欣悦 宗彦 林映冬 卢木仲 董林".split()
    data["审判程序s"] = ["民事提级管辖"]
    data["全文检索s"] = "华东科技 中国铁建 中国人寿 中国平安 中国建筑 中国中铁 牧原股份 中国石化 上汽集团 东风汽车集团 中国医药集团 中金公司 中国移动 工商银行 苏宁控股集团 太平洋建设集团".split()
    data["years"] = {'五个月': ['2022-10-15', '2023-03-15'], '十个月': ['2022-05-15', '2023-03-15'],'半年': ['2022-09-15', '2023-03-15'], '三年': ['2020-03-15', '2023-03-15'], '二年': ['2021-03-15', '2023-03-15'], '四年': ['2019-03-15', '2023-03-15']}
    data["裁判日期_1s"] = "2018-01-01 2019-01-01 2020-01-01 2021-01-01".split()
    data["裁判日期_2s"] = "2023-03-01 2023-09-01".split()
    wenshu_court_instruction_detail = []
    for i in range(1,1000000):
        adv = choice(advs)
        verb = choice(verbs)
        全文检索 = choice(data["全文检索s"])
        案由 = choice(data["案由s"])
        法院名称 = choice(data["法院名称s"])
        审判人员 = choice(data["审判人员s"])
        案件类型 = choice(data["案件类型s"])
        审判程序 = choice(data["审判程序s"])
        文书类型 = choice(data["文书类型s"])
        #year ,val = choice(data["years"]).items()
        year = choice(list(data["years"].keys()))
        val = data["years"][year]
        裁判日期_1 = choice(data["裁判日期_1s"])
        裁判日期_2 = choice(data["裁判日期_2s"])
        当事人 = choice(data["当事人s"])
        律师 = choice(data["律师s"])

        sample = {}
        key_value = {}
        instruct = adv + verb

        key_value["检索"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
        if(random.random()>0.5):
            key_value["全文检索"] = {'dom_type': '输入框', 'value': 全文检索, 'action': '输入'}
            instruct += 全文检索
        if(random.random()>0.5):
            key_value["当事人"] = {'dom_type': '输入框', 'value': 当事人, 'action': '输入'}
            instruct += "当事人是" + 当事人
        if(random.random()>0.5):
            key_value["律师"] = {'dom_type': '输入框', 'value': 律师, 'action': '输入'}
            instruct += "律师" + 律师
        if(random.random()>0.5):
            key_value["裁判日期_1"] = {'dom_type': '日期输入框', 'value': val[0], 'action': '输入'}
            key_value["裁判日期_2"] = {'dom_type': '日期输入框', 'value': val[1], 'action': '输入'}
            instruct += "最近" + year + "的"
        if(random.random() > 0.5 and year not in instruct):
            key_value["裁判日期_1"] = {'dom_type': '日期输入框', 'value': 裁判日期_1, 'action': '输入'}
            key_value["裁判日期_2"] = {'dom_type': '日期输入框', 'value': 裁判日期_2, 'action': '输入'}
            裁判日期_1 = datetime.strptime(裁判日期_1, "%Y-%m-%d").strftime("%Y年%m月%d日").replace('年0', '年').replace('月0', '月')
            裁判日期_2 = datetime.strptime(裁判日期_2, "%Y-%m-%d").strftime("%Y年%m月%d日").replace('年0', '年').replace('月0', '月')

            instruct += (裁判日期_1 + "到" + 裁判日期_2)

        if(random.random()>0.5):   
            key_value["案由"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
            key_value[案由] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
            instruct += "案由是" + 案由
        if(random.random()>0.5):
            key_value["法院名称"] = {'dom_type': '输入框', 'value': 法院名称, 'action': '输入'}
            instruct += 法院名称

        if(案由 not in instruct and random.random()>0.5):
            key_value["案件类型"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
            key_value[案件类型] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
            instruct += 案件类型
        if(random.random()>0.5 and random.random()>0.5):
            key_value["审判程序"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
            key_value[审判程序] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
            instruct += "审判程序为" + 审判程序

        key_value["文书类型"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
        key_value[文书类型] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
        instruct += 文书类型
        sample["instruction"] = instruct
        key_value["高级检索"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
        sample["key-value"] = key_value
        wenshu_court_instruction_detail.append(sample)
    wenshu_court_instruction_detail = random.sample(wenshu_court_instruction_detail, 10000)
    print(len(wenshu_court_instruction_detail))
    label_trainset[4]["instruction_detail"].extend(wenshu_court_instruction_detail)
    print(len(label_trainset[4]["instruction_detail"]))
test_1()

10000
10120


In [3]:
page_source = "学信网"
import random
from random import choice

def test_2():
    data ={}
    advs = ["", "请","帮"]
    verbs = ["查找","查询","搜索","检索","查一下","检索一下","查询一下","查一查","检索一下","查找出"]
    data["省市s"] = "青海省 河北省 甘肃省 北京市 湖北省 山西省 重庆市 云南省 浙江省 广西壮族自治区 新疆维吾尔自治区 天津市 陕西省 黑龙江省 西藏自治区 湖南省 上海市 内蒙古自治区".split()
    data["招生单位s"] = "西安交通大学 大连理工大学 兰州大学 山东大学 华东师范大学 北京师范大学 上海交通大学 华南理工大学 清华大学 华中科技大学 电子科技大学 中国海洋大学 同济大学 厦门大学 哈尔滨工业大学 复旦大学 四川大学 国防科学技术大学 中国农业大学 西北农林科技大学 北京理工大学".split()
    data["门类类别s"] = "哲学 经济学 法学 教育学 文学 历史学 理学 工学 农学 医学 军事学 管理学 艺术学 交叉学科 专业学位 专硕".split()
    data["学科类别s"] = "计算机应用技术 教育 食品科学 刑法学 档案学 植物学 国际关系 教育史 兽医 统计学 兽医学 地球化学 哲学 化学 基础医学 农业 口腔基础医学 药物化学 城乡规划管理 马克思主义哲学 军事学 计算数学 军事 工商管理 数学 新闻学 社会学 法学 教育学 基础兽医学 艺术".split()
    data["专业名称s"] = "国际关系 教育史 计算机应用技术 地球化学 基础兽医学 工商管理 食品科学 口腔基础医学 药物化学 统计学 城乡规划管理 刑法学 档案学 马克思主义哲学 计算数学 植物学 新闻学 社会学".split()
    data["学习方式s"] = "全日制 非全日制 ".split()

    chsi_instruction_detail = []
    for i in range(1, 1000000):
        adv = choice(advs)
        verb = choice(verbs)
        省市 = choice(data["省市s"])
        招生单位 = choice(data["招生单位s"])
        门类类别 = choice(data["门类类别s"])
        学科类别 = choice(data["学科类别s"])
        专业名称 = choice(data["专业名称s"])
        学习方式 = choice(data["学习方式s"])
        sample = {}
        key_value = {}
        instruct = ""

        key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
        instruct += adv + verb

        if(random.random()>0.5):
          key_value["招生单位"] = {'dom_type': '输入框', 'value': 招生单位, 'action': '输入'}
          instruct += 招生单位 + ","
        if(random.random()>0.5):
          key_value["门类类别"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
          key_value[门类类别] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
          if(门类类别 != "专业学位" and 门类类别 != "专硕"):
              instruct += 门类类别 + "学硕,"
          else:
              instruct += 门类类别 + ","
        if(random.random()>0.5):
            key_value["研究生院"] = {'dom_type': '复选框', 'value': '', 'action': '点击'}
            instruct += "招生单位有研究生院,"
        if(random.random()>0.5):
            key_value["博士点"] = {'dom_type': '复选框', 'value': '', 'action': '点击'}
            instruct += "招生单位有博士点,"
        if(random.random()>0.5):
            key_value["自划线院校"] = {'dom_type': '复选框', 'value': '', 'action': '点击'}
            instruct += "招生单位自划线院校,"

        if(random.random()>0.5):
          key_value["学科类别"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
          key_value[学科类别] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
          instruct += "学科类别是" + 学科类别 + ","
        if(random.random()>0.5):
          key_value["专业名称"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
          key_value[专业名称] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
          instruct +=  专业名称 + "专业,"
        if(random.random()>0.5):
          key_value["学习方式"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
          key_value[学习方式] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
          instruct += 学习方式 + ","

        instruct += "招生情况"
        sample["instruction"] = instruct
        sample["key-value"] = key_value
        chsi_instruction_detail.append(sample)
    chsi_instruction_detail = random.sample(chsi_instruction_detail, 10000)
    print(len(chsi_instruction_detail))
    label_trainset[13]["instruction_detail"].extend(chsi_instruction_detail)
    print(len(label_trainset[13]["instruction_detail"]))
test_2()

10000
10120


In [4]:
page_source = "中国法律服务网"
import random
from random import choice

def test_3():
    data ={}
    advs = ["", "请","帮"]
    verbs = ["查找","查询","搜索","检索","查一下","检索一下","查询一下","查一查","检索一下","查找出"]
    data["请输入律所关键字s"] = "致敦律师事务所 恒德律师事务所 多禾律师事务所 星权律师事务所 尤斯律师事务所 普光律师事务所 中仲律师事务所 泽仪律师事务所 易行律师事务所 存诚律师事务所 英舜律师事务所 京益律师事务所 金台律师事务所".split()
    data["请选择s"] = "安徽省 天津市 北京市 山西省 辽宁省 广东省 江西省 浙江省 吉林省 河南省 江苏省 山东省 福建省 黑龙江省 河北省 上海市 内蒙古自治区".split()
    data["单选框s"] = "门头沟区 怀柔区 昌平区 通州区 丰台区 西城区 石景山区 密云区 房山区 朝阳区 顺义区 东城区 延庆区".split()
    data["律师s"] = "刘威 卢木仲 陈欣悦 童康裕 陈义 董林 林映冬 洪嘉歆 郎辉 郑念雁".split()



    legal_service_instruction_detail = []
    for i in range(1, 1000000):
        adv = choice(advs)
        verb = choice(verbs)
        请输入律所关键字 = choice(data["请输入律所关键字s"])
        请选择 = choice(data["请选择s"])
        单选框 = choice(data["单选框s"])
        律师 = choice(data["律师s"])
        sample = {}
        key_value = {}
        instruct = ""
        key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
        instruct += adv + verb

        if(random.random()>0.5):
            key_value["请选择"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
            key_value[请选择] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
            instruct += 请选择

        if(random.random()>0.5):
            key_value[单选框] = {'dom_type': '单选框', 'value': '', 'action': '点击'}
            instruct += 单选框

        if(random.random()>0.5):
            key_value["请输入律所关键字"] = {'dom_type': '输入框', 'value': 请输入律所关键字, 'action': '输入'}
            instruct += 请输入律所关键字

        if(请输入律所关键字 in instruct):
            if(random.random()>0.5):
                key_value["请输入律师关键字"] = {'dom_type': '输入框', 'value': 律师, 'action': '输入'}
                instruct += 律师 + "律师"
        else:
            key_value["请输入律师关键字"] = {'dom_type': '输入框', 'value': 律师, 'action': '输入'}
            instruct += 律师 + "律师"

        instruct += "的信息"
        sample["instruction"] = instruct
        sample["key-value"] = key_value
        legal_service_instruction_detail.append(sample)

        sample = {}
        key_value = {}
        instruct = ""
        key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
        instruct += adv + verb

        if(random.random()>0.5):
            key_value["请选择"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
            key_value[请选择] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
            instruct += 请选择

        if(random.random()>0.5):
            key_value[单选框] = {'dom_type': '单选框', 'value': '', 'action': '点击'}
            instruct += 单选框
        key_value["律师"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
        instruct += "律师的"
        instruct += "信息"
        sample["instruction"] = instruct
        sample["key-value"] = key_value
        #print(sample)
        legal_service_instruction_detail.append(sample)
    legal_service_instruction_detail = random.sample(legal_service_instruction_detail, 10000)
    print(len(legal_service_instruction_detail))
    label_trainset[3]["instruction_detail"].extend(legal_service_instruction_detail)
    print(len(label_trainset[3]["instruction_detail"]))
test_3()

10000
10120


In [5]:
import random
from random import choice
page_source = "中国银行保险监督管理委员会"

def test_4():

    data ={}
    advs = ["", "请","帮"]
    verbs = ["查找","查询","搜索","检索","查一下","检索一下","查询一下","查一查","检索一下","查找出"]
    data["机构类型_1s"] = "人身险 其他 再保险 外资代表处 产险 资产管理 集团 控股".split()
    data["机构类型_2s"] = "专业责任险公司 专业车险公司 其他 出口信用险公司 政策性保险公司 专业农险公司".split()
    data["机构层级s"] = "分公司 营业部 营销服务部 中心支公司 专属机构(电销-省分) 省级分公司 支公司 专属机构(电销-总) 总公司".split()
    data["监管机构_1s"] = "天津银保监局 黑龙江银保监局 江苏银保监局 云南银保监局 广东银保监局 四川银保监局 陕西银保监局 上海银保监局 重庆银保监局 浙江银保监局 广西银保监局 辽宁银保监局 新疆银保监局 海南银保监局 青海银保监局 山西银保监局 吉林银保监局 银保监会 安徽银保监局 北京银保监局 湖北银保监局 贵州银保监局 河北银保监局".split()
    data["监管机构_2s"] = "深圳银保监分局 四平银保监分局 天津银保监局 江苏银保监局 湖南银保监局 广东银保监局 四川银保监局 陕西银保监局 邯郸银保监分局 上海银保监局 鞍山银保监分局 重庆银保监局 浙江银保监局 内蒙古银保监局 广西银保监局 广州银保监分局 三亚银保监分局 吉林银保监局 青岛银保监分局 宁夏银保监局 北京银保监局 贵州银保监局 河北银保监局".split()
    data["机构地址s"] = "广西壮族自治区南宁市青秀区桂春路13号 湖南省娄底市娄星区月塘街吉泰邦臣大厦A座 吉林省农安县合隆镇合隆街道小北庄585号的机构信息 吉林省农安县合隆镇合隆街道小北庄585号 德华安顾人寿保险有限公司泰安市新泰支公司 浙江省杭州市拱墅区天水街道中山北路中大广场A座7楼 河北省唐山市路北区唐山金融中心1号楼1405-1号 广西壮族自治区南宁市武鸣区城厢镇灵源7队唐家屯110号铺面 湖南省娄底市娄星区月塘街吉泰邦臣大厦A座的机构信息 河北省邯郸市北环路北东邢台村南".split()
    data["流水号s"] = "00115012 00009243 00089443 00115028 00070217 00022951 00009244 00115013 00110228".split()
    data["机构名称s"] = "华安财产保险股份有限公司梧州中心支公司 横琴人寿保险有限公司珠海市横琴支公司 合众人寿保险股份有限公司宜昌中心支公司枝江营销服务部 富德生命人寿保险股份有限公司武威中心支公司民勤支公司 富德财产保险股份有限公司 德华安顾人寿保险有限公司泰安市新泰支公司 国富人寿保险股份有限公司柳州分公司 合众人寿保险股份有限公司泰安中心支公司东平营销服务部 国富人寿保险股份有限公司柳州市柳江支公司的机构信息 国富人寿保险股份有限公司柳州市柳江支公司 都邦财产保险股份有限公司北仑支公司".split() #输入框


    regulatory_instruction_detail = []

    for i in range(1,1000000):
        adv = choice(advs)
        verb = choice(verbs)
        机构类型_1 = choice(data["机构类型_1s"])
        机构类型_2 = choice(data["机构类型_2s"])
        机构层级 = choice(data["机构层级s"])
        监管机构_1 = choice(data["监管机构_1s"])
        监管机构_2 = choice(data["监管机构_2s"])
        机构地址 = choice(data["机构地址s"])
        流水号 = choice(data["流水号s"])
        机构名称 = choice(data["机构名称s"])
        sample = {}
        key_value = {}
        instruct = ""

        key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
        instruct += adv + verb

        if(random.random()>0.5):
            key_value["监管机构_1"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
            key_value[监管机构_1] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
            instruct += "受监管于" + 监管机构_1 + ","
        if(random.random()>0.5):
            key_value["机构类型_1"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
            key_value[机构类型_1] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
            instruct += "机构类型为" + 机构类型_1 + ","

        if(random.random()>0.5):
            key_value["机构地址"] = {'dom_type': '输入框', 'value': 机构地址, 'action': '输入'}
            instruct += 机构地址
        if(random.random()>0.5):
            key_value["流水号"] = {'dom_type': '输入框', 'value': 流水号, 'action': '输入'}
            instruct += "流水号为"+ 流水号 + ","
        if(random.random()>0.5):
            key_value["机构名称"] = {'dom_type': '输入框', 'value': 机构名称, 'action': '输入'}
            instruct += "机构名为"+ 机构名称 + "机构信息"
        if(random.random()>0.5 and (监管机构_1 in instruct)):
            key_value["监管机构_2"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
            key_value[监管机构_2] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
            instruct += "受监管于" + 监管机构_2 + ","

        if(random.random()>0.5 and (机构类型_1 in instruct)):
            key_value["机构类型_2"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
            key_value[机构类型_2] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
            instruct += "机构类型为" + 机构类型_2 + ","
        if(random.random()>0.5):
            key_value["机构层级"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
            key_value[机构层级] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
            instruct += "属于" + 机构层级 + ","
        instruct = instruct.strip(",")
        instruct += "的机构信息"
        sample["instruction"] = instruct
        sample["key-value"] = key_value
        # print(sample)
        regulatory_instruction_detail.append(sample)
    regulatory_instruction_detail = random.sample(regulatory_instruction_detail, 10000)
    print(len(regulatory_instruction_detail))
    label_trainset[6]["instruction_detail"].extend(regulatory_instruction_detail)
    print(len(label_trainset[6]["instruction_detail"]))
test_4()

10000
10120


In [6]:
page_source = "qq音乐"
import random

def test_5():
    advs = ["", "请", "帮助", "我要"]
    verbs = ["查找", "查询", "查找一下", "查询一下", "查一查"]
    countries = ["", "内地", "港台", "欧美", "日本", "韩国"]
    styles = ["", "流行", "说唱", "国风", "摇滚", "电子", "民谣", "轻音乐", "爵士", "古典", "乡村", "蓝调", "R&B", "民族乐"]
    sexs = ["", "男", "女", "组合"]

    qq_music_instruction_detail = []

    qq_web_element = '''
    可以[点击]名称为[搜索]的[按钮]。可以[点击]名称为[内地 港台 欧美 日本 韩国]的[单选框]。可以[点击]名称为[男 女 组合]的[单选框]。可以[点击]名称为[流行 说唱 国风 摇滚 电子 民谣 轻音乐 爵士 古典 乡村 蓝调 R&B 民族乐]的[单选框]。
    '''

    generate_train1 = []
    generate_train2 = []
    generate_train3 = []

    for adv in advs:
        for verb in verbs:
            for country in countries:
                for sex in sexs:
                    for style in styles:
                        sample = {}
                        actions = [country, style, sex]
                        actions = list(filter(lambda x:len(x)>0, actions))
                        Country = country + "地区的" if len(country) > 0 else country
                        Style = style + "风格的" if len(style) > 0 else style
                        if sex == "组合":
                            Sex = sex + "形式的"
                        elif sex == "男" or sex == "女":
                            Sex  = sex + "性的"
                        else:
                            Sex = sex
                        select_rand = random.randint(0, 1)
                        if select_rand % 2 == 0:
                            instruct = adv + verb + Country + Style + Sex + "歌手列表"
                        else:
                            instruct = adv + verb + Sex + Country + Style + "歌手列表"

                        sample["instruction"] = instruct
                        key_value = {}
                        key_value["搜索"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                        for action in actions:
                            key_value[action] = {'dom_type': '单选框', 'value': '', 'action': '点击'}
                        sample["key-value"] = key_value
                        # print(sample)
                        qq_music_instruction_detail.append(sample)
    print(len(qq_music_instruction_detail))
    label_trainset[0]["instruction_detail"].extend(qq_music_instruction_detail)
    print(len(label_trainset[0]["instruction_detail"]))
test_5()

6720
6840


In [7]:
page_source = "中国证券投资基金业协会"
import random
from random import choice
from datetime import datetime
def test_6():
    advs = ["", "请"]
    verbs = ["查找","查询","搜索","检索","查一下","检索一下","查询一下","查一查","检索一下","查找出"]
    data = {}
    data["机构名称s"] = "新华基金管理股份有限公司 建信基金管理有限责任公司 华泰柏瑞基金管理有限公司 国海富兰克林基金管理有限公司 汇添富基金管理股份有限公司".split()
    data["会员编号s"] = "PT0100000022 PT0100000024 PT0100000025".split()
    data["会员代表s"] = "王鸿嫔 杨仓兵 黄小薏".split()
    data["years"] = {'五个月': ['2022-10-15', '2023-03-15'], '十个月': ['2022-05-15', '2023-03-15'],'半年': ['2022-09-15', '2023-03-15'], '三年': ['2020-03-15', '2023-03-15'], '二年': ['2021-03-15', '2023-03-15'], '四年': ['2019-03-15', '2023-03-15'], '六年': ['2017-03-15', '2023-03-15']}
    data["入会时间_1s"] = "2018-02-10 2021-01-01".split()
    data["入会时间_2s"] = "2022-08-20 2023-09-01".split()

    data["会员类型s"] = ["全部会员类型",
                 "普通会员", 
                 "联席会员",
                 "观察会员", 
                 "特别会员", 
                 "普通会员和特别会员", 
                 "联席会员和观察会员",
                 "普通会员、联席会员和观察会员", 
                 "联席会员、观察会员和特别会员"]   # 会员类型 
    data["机构类型s"] = ["全部机构类型", 
              "公募基金管理公司",
              "私募基金管理人",
              "证劵公司",
              "私募基金管理人和证劵公司"
              "商业银行",
              "公募基金管理公司子公司",
              "证劵公司资管子公司",
              "期货公司资管子公司",
              "保险公司子公司",
              "独立第三方销售机构",
              "会计师事务所",
              "证劵投资咨询机构",
              "独立服务机构",
              "境外机构",
              "地方自律组织",
              "其他机构",
              "保险公司和期货银行",
              "会计师事务所和独立服务机构",
              "证劵公司资管子公司和期货公司资管子公司",
              "证券公司、证券投资咨询机构和私募基金管理人"]  # 机构类型

    advs = ["", "请", "帮助", "我要", "帮我"]
    verbs = ["查找", "查询", "查找一下", "查询一下", "查一查"]

    asset_china_instruction_detail = []




    for i in range(1,1000000):
        adv = choice(advs)
        verb = choice(verbs)

        机构名称 = choice(data["机构名称s"])
        会员编号 = choice(data["会员编号s"])
        会员代表 = choice(data["会员代表s"])



        会员类型_1 = choice(data["会员类型s"])
        会员类型_2 = choice(data["会员类型s"])
        会员类型_3 = choice(data["会员类型s"])

        机构类型_1 = choice(data["机构类型s"])
        机构类型_2 = choice(data["机构类型s"])
        机构类型_3 = choice(data["机构类型s"])

        入会时间_1 = choice(data["入会时间_1s"])
        入会时间_2 = choice(data["入会时间_2s"])

        year = choice(list(data["years"].keys()))
        val = data["years"][year]

        sample = {}
        key_value = {}

        instruct = adv + verb

        key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}



        if(random.random()>0.5):
            key_value["入会时间_1"] = {'dom_type': '日期输入框', 'value': val[0], 'action': '输入'}
            key_value["入会时间_2"] = {'dom_type': '日期输入框', 'value': val[1], 'action': '输入'}
            if(random.random()>0.5):
                instruct += "最近" + year + "之内"
            else:
                instruct += "最近" + year + "的"
        if(random.random() > 0.5 and year not in instruct):
            key_value["入会时间_1"] = {'dom_type': '日期输入框', 'value': 入会时间_1, 'action': '输入'}
            key_value["入会时间_2"] = {'dom_type': '日期输入框', 'value': 入会时间_2, 'action': '输入'}
            入会时间_1 = datetime.strptime(入会时间_1, "%Y-%m-%d").strftime("%Y年%m月%d日").replace('年0', '年').replace('月0', '月')
            入会时间_2 = datetime.strptime(入会时间_2, "%Y-%m-%d").strftime("%Y年%m月%d日").replace('年0', '年').replace('月0', '月')

            instruct += (入会时间_1 + "到" + 入会时间_2 + "期间的")


        if(random.random()>0.5):
            key_value["机构名称"] = {'dom_type': '输入框', 'value': 机构名称, 'action': '输入'}
            instruct += 机构名称
        if(random.random()>0.5 and 机构名称 not in instruct):
            key_value["会员编号"] = {'dom_type': '输入框', 'value': 会员编号, 'action': '输入'}
            instruct += "会员编号为" + 会员编号
        if(random.random()>0.5 and 机构名称 not in instruct and 会员编号 not in instruct):
            key_value["会员代表"] = {'dom_type': '输入框', 'value': 会员代表, 'action': '输入'}
            instruct += "会员代表" + 会员代表


        if(random.random()>0.5):   
            key_value["会员类型"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
            key_value[会员类型_1] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
            instruct += 会员类型_1
            if(random.random()>0.5 and 会员类型_1 != 会员类型_2):   
                key_value[会员类型_2] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                instruct += "和" + 会员类型_2
            if(random.random()>0.5 and 会员类型_1 != 会员类型_3 and 会员类型_2 != 会员类型_3):
                key_value[会员类型_3] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                instruct += "和" + 会员类型_3
        if(random.random()>0.5):   
            key_value["机构类型"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
            key_value[机构类型_1] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
            instruct += 机构类型_1
            if(random.random()>0.5 and 机构类型_1 != 机构类型_2):   
                key_value[机构类型_2] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                instruct += "和" + 机构类型_2
            if(random.random()>0.5 and 机构类型_2 != 机构类型_3 and 机构类型_1 != 机构类型_3):
                key_value[机构类型_3] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                instruct += "和" + 机构类型_3

        sample["instruction"] = instruct + "的信息"

        key_value["关闭"] = {'dom_type': '按钮', 'value': '', 'action': '点击'} 
        sample["key-value"] = key_value
        asset_china_instruction_detail.append(sample)
    asset_china_instruction_detail = random.sample(asset_china_instruction_detail, 10000)                                      
    print(len(asset_china_instruction_detail))
    label_trainset[5]["instruction_detail"].extend(asset_china_instruction_detail)
    print(len(label_trainset[5]["instruction_detail"]))
test_6()

10000
10120


In [8]:
page_source = "中国人民银行"
import random
from random import choice
from datetime import datetime

def test_7():
    advs = ["", "请"]
    verbs = ["查找","查询","搜索","检索","查一下","检索一下","查询一下","查一查","检索一下","查找出"]
    data = {}
    data["包含以下全部的关键词s"] = "基金 征信 汇率 货币 人民币 反洗钱 国际贸易 石油价格 股票涨势 金融市场".split()
    data["包含以下完整关键词s"] = "基金 征信 汇率 货币 反洗钱 国际贸易 石油价格 股票涨势 金融市场".split()
    data["包含以下任意一个关键词s"] = "基金 征信 汇率 货币 人民币 反洗钱 国际贸易 石油价格 股票涨势 金融市场".split()
    data["不包含以下关键词s"] = "跌破 失业率 亚洲市场 势头良好 市场干扰 金融风暴".split()
    data["请选择查询范围s"] = "上海总部 调查统计司 中国人民银行 全站 党委宣传部 长春中心支行 出版物 天津分行".split()
    data["限定要搜索的内容的时间是s"] = "时间不限 一天内 一周内 一月内 一年内".split()
    data["查询关键词位于s"] = "默认 标题".split()

    people_bank_instruction_detail = []

    for i in range(1,1000000):
        adv = choice(advs)
        verb = choice(verbs)

        包含以下全部的关键词 = choice(data["包含以下全部的关键词s"])
        包含以下完整关键词 = choice(data["包含以下完整关键词s"])
        包含以下任意一个关键词 = choice(data["包含以下任意一个关键词s"])
        不包含以下关键词 = choice(data["不包含以下关键词s"])

        请选择查询范围 = choice(data["请选择查询范围s"])
        限定要搜索的内容的时间是 = choice(data["限定要搜索的内容的时间是s"])
        查询关键词位于 = choice(data["查询关键词位于s"])

        sample = {}
        key_value = {}
        key_value['高级搜索'] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
        instruct = adv + verb
        if (random.random() > 0.5):
            key_value['包含以下全部的关键词'] = {'dom_type': '输入框', 'value': 包含以下全部的关键词, 'action': '输入'}
            instruct += "包含全部关键词如" + "”" + 包含以下全部的关键词 + "”,"
        if (random.random() > 0.5):
            key_value['包含以下完整关键词'] = {'dom_type': '输入框', 'value': 包含以下完整关键词, 'action': '输入'}
            instruct += "包含完整关键词" + "”" + 包含以下完整关键词 + "”,"
        if (random.random() > 0.5):
            key_value['包含以下任意一个关键词'] =  {'dom_type': '输入框', 'value': 包含以下任意一个关键词, 'action': '输入'}
            instruct += "包含某个关键词" + "”" + 包含以下任意一个关键词 + "”,"
        if (random.random() > 0.5):
            key_value['不包含以下关键词'] =  {'dom_type': '输入框', 'value': 不包含以下关键词, 'action': '输入'}
            instruct += "不包含关键词如" + "”" + 不包含以下关键词 + "”,"
        if (random.random() > 0.5):
            key_value[查询关键词位于] = {'dom_type': '单选框', 'value': '', 'action': '点击'}
            instruct += "关键词在" + 查询关键词位于 + "位置的"
        if (random.random() > 0.5):
            key_value['请选择查询范围'] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
            key_value[请选择查询范围] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
            instruct += "”" + 请选择查询范围 + "“范围内查询的"
        if (random.random() > 0.5):
            key_value[限定要搜索的内容的时间是] = {'dom_type': '单选框', 'value': '', 'action': '点击'}
            instruct += 限定要搜索的内容的时间是 + "的"
        instruct += "查询结果"
        sample["instruction"] = instruct
        sample["key-value"] = key_value
        people_bank_instruction_detail.append(sample)
    people_bank_instruction_detail = random.sample(people_bank_instruction_detail, 10000)                                      
    print(len(people_bank_instruction_detail))
    label_trainset[2]["instruction_detail"].extend(people_bank_instruction_detail)
    print(len(label_trainset[2]["instruction_detail"]))
test_7()

10000
10120


In [9]:
page_source = "猫眼电影"
import random
from random import choice
from datetime import datetime

def test_8():
    advs = ["", "请"]
    verbs = ["查找","查询","搜索","检索","查一下","检索一下","查询一下","查一查","检索一下","查找出"]
    data = {}
    data["类型s"] = "动作 其他 科幻 青春 戏曲 短片 恐怖 纪录片 家庭 黑色电影 战争 音乐 武侠 惊悚 动画 悬疑 灾难 西部 儿童".split()
    data["区域s"] = "日本 法国 西班牙 伊朗 其他 俄罗斯 澳大利亚 德国 印度 美国 英国 大陆 波兰 泰国".split()
    data["年代s"] = "2013 2012 2017 2020 70年代 2011 2014 2018 2000-2010 2015 更早 2019 2023 2021".split()
    data["排序方式s"] = "按热门排序 按评价排序 按时间排序".split()


    maoyan_instruction_detail = []

    for i in range(1,1000000):
        adv = choice(advs)
        verb = choice(verbs)

        类型 = choice(data["类型s"])
        区域 = choice(data["区域s"])
        年代 = choice(data["年代s"])
        排序方式 = choice(data["排序方式s"])

        sample = {}
        key_value = {}
        key_value['检索'] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
        instruct = adv + verb
        if (random.random() > 0.5):
            key_value['类型'] = {'dom_type': '单选框', 'value': '', 'action': '点击'}
            instruct += 类型 + "类的"
        if (random.random() > 0.5):
            key_value['排序方式'] =  {'dom_type': '单选框', 'value': '', 'action': '点击'}
            instruct += "按" + 排序方式 + "的"
        if (random.random() > 0.5):
            key_value['区域'] = {'dom_type': '单选框', 'value': '', 'action': '点击'}
            instruct += 区域 + "地区的"
        if (random.random() > 0.5):
            key_value['年代'] =  {'dom_type': '单选框', 'value': '', 'action': '点击'}
            if("年" not in instruct):
                instruct += 年代 + "年的"
            else:
                instruct += 年代
        instruct += "的电影"
        sample["instruction"] = instruct
        sample["key-value"] = key_value
        maoyan_instruction_detail.append(sample)
    maoyan_instruction_detail = random.sample(maoyan_instruction_detail, 10000)                                      
    print(len(maoyan_instruction_detail))
    label_trainset[17]["instruction_detail"].extend(maoyan_instruction_detail)
    print(len(label_trainset[17]["instruction_detail"]))
test_8()

10000
10120


In [10]:
page_source = "巨潮资讯"
import random
from random import choice
from datetime import datetime

def test_9():
    advs = ["", "请"]
    verbs = ["查找","查询","搜索","检索","查一查","查一下","检索一下","查询一下","查查","检索一下","查找出"]
    data = {}
    data["板块s"] = "深户京 基金 债券 港股 三板".split()
    data["开始日期s"] = "2021-01-01 2022-02-10 ".split()
    data["结束日期s"] = "2023-01-01 2023-03-20".split()
    data["years"] = {'五个月': ['2022-10-15', '2023-03-15'], '十个月': ['2022-05-15', '2023-03-15'],'一年': ['2022-03-15', '2023-03-15'], '三年': ['2020-03-15', '2023-03-15'], '二年': ['2021-03-15', '2023-03-15']}

    data["代码/简称/拼音s"] = "000009 000011 000002 000007 000010 000006 000008 000005".split()
    data["标题关键字s"] = "ST国华 深物业A 神州高铁 平安银行 ST星源 深振业A 美丽生态 中国宝安 万科A 全新好".split()
    data["深户京s"] = "澄清致歉 中介报告 配股 其他融资 股东大会 风险提示 股权变动 特别处理和退市 三季报 退市整理期 年报 董事会 权益分派 增发 一季报 首发 公司债".split()
    data["基金s"] = "基金费率 申购赎回 中报 分红 其他 投资组合 季报 基本信息变更 年报 持有人大会 说明书更新 高管及基金经理 招募设立 销售渠道".split() #+ [""]
    data["债券s"] = "债券付息公告 债券发行上市 债券定期公告 债券到期兑付停".split() #+ [""]
    data["三板s"] = "中介机构公告 首次信息披露 定期公告 持续信息披露 临时公告".split() #+ [""]


    巨潮查询_instruction_detail = []

    for i in range(1,1000000):
        adv = choice(advs)
        verb = choice(verbs)

        板块 = choice(data["板块s"])

        代码 = choice(data["代码/简称/拼音s"])
        标题关键字 = choice(data["标题关键字s"])


        深户京_1 = choice(data["深户京s"])
        基金_1 = choice(data["基金s"])
        债券_1 = choice(data["债券s"])
        三板_1 = choice(data["三板s"])

        深户京_2 = choice(data["深户京s"])
        基金_2 = choice(data["基金s"])
        债券_2 = choice(data["债券s"])
        三板_2 = choice(data["三板s"])

        year = choice(list(data["years"].keys()))
        val = data["years"][year]

        开始日期 = choice(data["开始日期s"])
        结束日期 = choice(data["结束日期s"]) 


        sample = {}
        key_value = {}
        key_value['查询'] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
        instruct = adv + verb
        key_value[板块] = {'dom_type': '按钮', 'value': '', 'action': '点击'}

        if(random.random()>0.5):
            key_value["标题关键字"] = {"dom_type": "输入框", "value": 标题关键字, "action": "输入"}
            instruct += 标题关键字
        if(random.random()>0.5 and 标题关键字 not in instruct):
            key_value['代码/简称/拼音'] = {"dom_type": "输入框", "value": 代码, "action": "输入"}
            instruct += "代码为" + 代码 + "公司的"
        if(random.random()>0.5):
            key_value['开始日期'] = {"dom_type": "日期输入框", "value": 开始日期, "action": "输入"} # ???
            key_value['结束日期'] = {"dom_type": "日期输入框", "value": 结束日期, "action": "输入"} # ???
            开始日期 = datetime.strptime(开始日期, "%Y-%m-%d").strftime("%Y年%m月%d日").replace('年0', '年').replace('月0', '月')
            结束日期 = datetime.strptime(结束日期, "%Y-%m-%d").strftime("%Y年%m月%d日").replace('年0', '年').replace('月0', '月')
            instruct += 开始日期 + "到" + 结束日期 + "期间的"
        if(random.random()>0.5 and "开始日期" not in instruct):
            key_value['开始日期'] = {"dom_type": "日期输入框", "value": val[0], "action": "输入"} # ???
            key_value['结束日期'] = {"dom_type": "日期输入框", "value": val[1], "action": "输入"} # ???
            instruct +=  "最近" + year

        if(板块 == "深户京"):
            if(random.random()>0.5):
                key_value["分类"] = {"dom_type": "下拉框", "value": "", "action": "点击"}
                key_value[深户京_1] = {"dom_type": "下拉框选项", "value": "", "action": "点击"}
                instruct += "深户京的" +  深户京_1
                if(random.random()>0.5):
                    key_value[深户京_2] = {"dom_type": "下拉框选项", "value": "", "action": "点击"}
                    instruct += "和" +  深户京_2
            else:
                instruct += "深户京的公告" 

        elif(板块 == "基金"):
            if(random.random()>0.5):
                key_value["分类"] = {"dom_type": "下拉框", "value": "", "action": "点击"}
                key_value[基金_1] = {"dom_type": "下拉框选项", "value": "", "action": "点击"}
                instruct += "基金的" + 基金_1
                if(random.random()>0.5):
                    key_value[基金_2] = {"dom_type": "下拉框选项", "value": "", "action": "点击"}
                    instruct += "和" +  基金_2
            else:
                instruct += "基金的公告" 
        elif(板块 == "债券"):
            if(random.random()>0.5):
                key_value["分类"] = {"dom_type": "下拉框", "value": "", "action": "点击"}
                key_value[债券_1] = {"dom_type": "下拉框选项", "value": "", "action": "点击"}
                instruct += 债券_1
                if(random.random()>0.5):
                    key_value[债券_2] = {"dom_type": "下拉框选项", "value": "", "action": "点击"}
                    instruct += "和" +  债券_2
            else:
                instruct += "债券的公告"
        elif(板块 == "三板"):
            if(random.random()>0.5):
                key_value["分类"] = {"dom_type": "下拉框", "value": "", "action": "点击"}
                key_value[三板_1] = {"dom_type": "下拉框选项", "value": "", "action": "点击"}
                instruct += "三板" +  三板_1
                if(random.random()>0.5):
                    key_value[三板_2] = {"dom_type": "下拉框选项", "value": "", "action": "点击"}
                    instruct += "和" +  三板_2
            else:
                instruct += "三板的公告"
        elif(板块 == "港股"):
            instruct += "公告"
        key_value['确认'] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
        sample["instruction"] = instruct
        sample["key-value"] = key_value
        巨潮查询_instruction_detail.append(sample)
    巨潮查询_instruction_detail = random.sample(巨潮查询_instruction_detail, 10000)                                      
    print(len(巨潮查询_instruction_detail))
    label_trainset[15]["instruction_detail"].extend(巨潮查询_instruction_detail)
    print(len(label_trainset[15]["instruction_detail"]))
test_9()

10000
10120


In [11]:
def encode_instruct(web, instruct):
    desc = "用户的需求是[%s]。请问在网页上执行什么操作？" % (instruct)
    return desc


def encode_response(web, instruct, key_values):
    desc = "用户使用的网站是[%s]。在网页上" % (web)
    for key, value in key_values.items():
        action = ""
        # print(key, value)
        if value["action"] == "点击":
            action = "[点击]名称为[%s]的[%s]" % (key, value["dom_type"])
        elif value["action"] == "输入":
            action = "[输入]值[%s]进名称为[%s]的[%s]" % (value["value"], key, value["dom_type"])
        desc += action + ";"
    return desc
        

def decode_response(desc):
    response = {}
    actions = desc.split("。")[1]
    regex = r"(\[.*?\])"
    for action in actions.split(";"):
        matches = re.findall(regex, action)
        if len(matches) == 4:
            response[matches[2][1:-1]] = {'dom_type':matches[3][1:-1],
                                          'value':matches[1][1:-1],
                                          'action':'输入'}
        elif len(matches) == 3:
            response[matches[1][1:-1]] = {'dom_type':matches[2][1:-1],
                                          'value':'',
                                          'action':'点击'}
    return response

chat_data = []
for idx in range(0, 20):
    if(len(label_trainset[idx]['instruction_detail']) < 2000):
        print(idx)
        continue
    for instructions in label_trainset[idx]['instruction_detail']:
        chat_data.append({
            'prompt':  encode_instruct(label_trainset[idx]["page_source"], instructions["instruction"]),
            'response': encode_response(label_trainset[idx]["page_source"], instructions["instruction"], instructions['key-value']),
            "history": []
        })
random.shuffle(chat_data)
print(len(chat_data))
train_len = 70000
df_train = chat_data[:train_len]
df_valid = chat_data[train_len:]  # 剩下的就是valid
print(len(df_train))
print(len(df_valid))
with open('./data_2/train_9.json', 'w') as up:
    for line in df_train:
        up.write(json.dumps(line, ensure_ascii=False)+'\n')
with open('./data_2/dev_9.json', 'w') as up:
    for line in df_valid:
        up.write(json.dumps(line,  ensure_ascii=False)+'\n')

1
7
8
9
10
11
12
14
16
18
19
87800
70000
17800


In [12]:
max_value = 0
sum = 0
cnt = 0
delete_page = [1,7,8,9,10,11,12,14,16,18,19]
for line in df_train:
    t = int(len(line["prompt"]))
    if(t > 128):
         cnt = cnt+1
    sum = sum + t
print(sum/len(df_train))
print(cnt)

56.38164285714286
243


In [13]:
def encode_instruct(web, instruct):
    desc = "用户的需求是[%s]。请问在网页上执行什么操作？" % (instruct)
    return desc
delete_page = [1,7,8,9,10,11,12,14,16,18,19]
test_data = []
for idx in range(0, 20):
    if(idx in delete_page):
        continue
    print(idx)
    print(instruction_testB[idx]["page_source"])
    print(len(instruction_testB[idx]["instruction"]))
    
    for instructions in instruction_testB[idx]['instruction']:
        test_data.append({
            'prompt':  encode_instruct(instruction_testB[idx]["page_source"], instructions)
        })
with open('./data_2/test_9.json', 'w') as up:
    for line in test_data:
        up.write(json.dumps(line,  ensure_ascii=False)+'\n')
    

0
qq音乐
50
2
中国人民银行
50
3
中国法律服务网
50
4
中国裁判文书网
50
5
中国证券投资基金业协会
50
6
中国银行保险监督管理委员会
50
13
学信网
50
15
巨潮资讯
50
17
猫眼电影
50


In [14]:
# 读取数据集
LABEL_PATH = './data_2/train/'
submission_20 = json.load(open(LABEL_PATH + 'submission_20.json',encoding='utf8'))
submission_9 = json.load(open(LABEL_PATH + 'submission_20.json',encoding='utf8'))

In [15]:
delete_page = [1,7,8,9,10,11,12,14,16,18,19]
for idx in range(0, 20):
    if(idx not in delete_page):
        continue
    print(idx)
    print(submission_20[idx]["page_source"])

1
上交所
7
交通安全综合服务管理平台
8
企查查企业查询
9
卫健委信用信息网_医
10
卫健委信用信息网_护
11
国家企业信用信息公示系统
12
国家统计局
14
安居客
16
火车票12306
18
顺丰快递查收寄标准
19
顺丰快递查运费时效


In [16]:
# pages = ["qq音乐",
# "中国人民银行",
# "中国法律服务网",
# "中国裁判文书网",
# "中国证券投资基金业协会",
# "中国银行保险监督管理委员会",
# "学信网",
# "巨潮资讯",
# "猫眼电影"]

pages = ["qq音乐",
"中国人民银行",
"中国法律服务网",
"中国裁判文书网",
"中国证券投资基金业协会",
"中国银行保险监督管理委员会"]
new_page = []
for idx in range(0, 10):
    if(submission_9[idx]["page_source"] not in pages):
        continue
    # print(submission_9[idx]["page_source"])
    new_page.append(submission_9[idx]["page_source"])
print(' '.join(new_page))

new_sub = []

for idx in range(0, 20):
    if(submission_20[idx]["page_source"] in pages):
        continue
    print(idx)
    print(submission_20[idx]["page_source"])

    new_sub.append({
        'page_source': submission_20[idx]["page_source"],
        'instruction_detail': submission_20[idx]["instruction_detail"]
    })
for sub in submission_9:
    if(sub["page_source"] in pages):
        new_sub.append({
        'page_source': sub["page_source"],
        'instruction_detail': sub["instruction_detail"]
        })

qq音乐 中国人民银行 中国法律服务网 中国裁判文书网 中国证券投资基金业协会 中国银行保险监督管理委员会
1
上交所
7
交通安全综合服务管理平台
8
企查查企业查询
9
卫健委信用信息网_医
10
卫健委信用信息网_护
11
国家企业信用信息公示系统
12
国家统计局
13
学信网
14
安居客
15
巨潮资讯
16
火车票12306
17
猫眼电影
18
顺丰快递查收寄标准
19
顺丰快递查运费时效


In [17]:
for idx, sub in enumerate(new_sub):
    print(idx)
    print(sub["page_source"])
with open('new_submission.json', 'w') as up:
    json.dump(new_sub, up,ensure_ascii=False)

0
上交所
1
交通安全综合服务管理平台
2
企查查企业查询
3
卫健委信用信息网_医
4
卫健委信用信息网_护
5
国家企业信用信息公示系统
6
国家统计局
7
学信网
8
安居客
9
巨潮资讯
10
火车票12306
11
猫眼电影
12
顺丰快递查收寄标准
13
顺丰快递查运费时效
14
qq音乐
15
中国人民银行
16
中国法律服务网
17
中国裁判文书网
18
中国证券投资基金业协会
19
中国银行保险监督管理委员会
